In [1]:
import sys
sys.path.append("../src")

In [2]:
from pathlib import Path
from collections import defaultdict

from gensim.corpora import MmCorpus, Dictionary
from gensim.models.ldamodel import LdaModel

from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, Float
from sqlalchemy import create_engine, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import pyLDAvis.gensim

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/_pytest/mark/structures.py:380: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import db

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
import imp; imp.reload(db)

<module 'db' from '../src/db.py'>

In [5]:
prepared_data_dir = Path("../scratch/prepared_data/")

In [6]:
# corpus = MmCorpus('../scratch/prepared_data/corpus.mm')
# dictionary = Dictionary.load("../scratch/prepared_data/dct.mm")
corpus, dictionary, corp2paper, dct2kwd = db.read_from_prepared_data(prepared_data_dir)
mdir = Path('../scratch/tmodels/')
n_topics = 20
tmodel_loc = mdir / f'topic_model{n_topics}'

lda_model = LdaModel.load(str(tmodel_loc))
tm = db.TopicModeler(dictionary, corpus)
embedding = tm.get_inference(lda_model)
coh_per_topic = tm.get_coherence_model(lda_model).get_coherence_per_topic()

100%|██████████| 6869/6869 [00:02<00:00, 2923.49it/s]


In [7]:
# from topic_modeling import NumPyEncoder, __num_dist_rows__

In [10]:
# def __num_dist_rows__(array, ndigits=2):
#     return array.shape[0] - int((pd.DataFrame(array).sum(axis=1) < 0.9).sum())

In [14]:
pyLDAvis._prepare.__num_dist_rows__(np.matrix(embedding))

6869

In [15]:
# (pd.DataFrame(embedding).sum(axis=1) < 0.999).sum()

In [22]:
# pyLDAvis.utils.NumPyEncoder = NumPyEncoder
# pyLDAvis._prepare.__num_dist_rows__ = __num_dist_rows__

In [27]:
# imp.reload(pyLDAvis.gensim)
# imp.reload(pyLDAvis)b
lda_model.id2word

In [ ]:
viz_data = pyLDAvis.gensim.prepare(lda_model, tm.corpus, lda_model.id2word, doc_topic_dist=np.matrix(embedding), sort_topics=False, mds="mmds", start_index=0)

> /Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/pyLDAvis/_prepare.py(64)_input_check()
     63     ipdb.set_trace()
---> 64     if __num_dist_rows__(topic_term_dists) != ttds[0]:
     65         err('Not all rows (distributions) in topic_term_dists sum to 1.')



In [31]:
len(lda_model.id2word)

1630

In [32]:
len(tm.dictionary)

1580

In [25]:
db_loc = '../scratch/test.sqlite'

In [26]:
engine = create_engine(f"sqlite:///{db_loc}")
Session = sessionmaker(bind=engine)
session = Session()

In [27]:
session.query(db.Paper.id).count()

7245

In [28]:
corp2paper_dct = {c: p for c, p in corp2paper}
dct2kwd_dct = {d: k for d, k in dct2kwd}

In [141]:
corpus_inds, paper_inds = zip(*corp2paper)

In [95]:
paper2bibcode = session.query(db.Paper.id, db.Paper.bibcode).all()

In [96]:
# q = (
#     session.query(db.Paper.bibcode)
#     .filter(db.Paper.id.in_([p for _, p in corp2paper]))
# )
# bibs = [q[0] for q in q.all()]

In [100]:
# bibs = [[c, paper2bibcode_dct[p]] for c, p in corp2paper]

In [140]:
bibs = [paper2bibcode_dct[p] for c, p in corp2paper]

In [294]:
embedding_df = pd.DataFrame(embedding)
embedding_df.index = paper_inds

# Get Topic Years

In [313]:
from tsfresh import extract_features
import numpy as np

In [367]:
thresh = 0.3

In [368]:
year_min = 1997
year_max = 2010

In [369]:
all_time_series = []
for topic in tqdm(embedding_df.columns):
    ids_in_topic = embedding_df.index[embedding_df.loc[:, topic] > thresh].tolist() # function to include options with argmax as well?
    years_query = (
        session.query(db.Paper.year, func.count(db.Paper.year)) # Don't really need to add journal exlusions because already limiting to the IDs which are in topic model
        .filter(db.Paper.id.in_(ids_in_topic))
        .filter(db.Paper.year <= year_max)
        .filter(db.Paper.year >= year_min)
        .group_by(db.Paper.year)
    )
    year_counts = years_query.all()
    ycd = defaultdict(int, {y:c for y, c in year_counts})
    topic_time_series = [{"topic": topic, "year": y, "count": ycd[y]} for y in range(year_min, year_max)]
    all_time_series = all_time_series + topic_time_series

ts_df_long = pd.DataFrame(all_time_series)
ts_df = ts_df_long.pivot(index='topic', columns="year", values="count")

features_df = extract_features(ts_df_long, column_id='topic', column_sort='year')
features_df['coherence_score'] = coh_per_topic

def cagr(x_row):
    x = x_row.values
    nz_inds = np.nonzero(x)[0]
    if len(nz_inds) == 0:  # If all are 0, set CAGR to 0
        return 0
    else:
        first_nonzero_index = nz_inds[0]
        x = x[first_nonzero_index:]  # Not valid if starts with 0. Becomes inf
        x = x[~np.isnan(x)]  # For normalized time series, NaNs before any occurrence of kwd
    if len(x) < 2:  # If no periods, set CAGR to 0
        return 0
    else:
        ys = x_row.index
        period = max(ys) - min(ys)
        return (x[-1] / x[0]) ** (1 / period) - 1

features_df['CAGR'] = ts_df.apply(cagr, axis=1)


Feature Extraction: 100%|██████████| 20/20 [00:00<00:00, 108.62it/s]A


In [419]:
import dtw_time_analysis as dtw
import imp; imp.reload(dtw)

<module 'dtw_time_analysis' from '../src/dtw_time_analysis.py'>

In [420]:
dtw_df = dtw.dtw_kwds(ts_df)

INFO:dtw_time_analysis:Computing dynamic time warping between keywords.
INFO:dtw_time_analysis:window: 1.
